In [1]:
import os
import shutil
import random
import cv2
from tqdm import tqdm
import numpy as np

### 0. Configuration & Setup

In [2]:
RAW_VIDEO_DATASET_PATH = "SignLanguageDataset"  # <--- UPDATE THIS PATH to your raw videos
BASE_OUTPUT_PATH_FRAMES = "SignLanguage_Processed_Frames" # Root for frame-based processing
YOLO_FRAME_DATASET_ROOT = os.path.join(BASE_OUTPUT_PATH_FRAMES, "YOLO_Frames_Dataset") # Output for YOLOv8 cls

# Video Splitting Parameters
VIDEO_SPLIT_PATH = os.path.join(BASE_OUTPUT_PATH_FRAMES, "video_splits_for_frames")
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15 # YOLOv8 training will use this as its validation set
TEST_RATIO = 0.15 # This will be a hold-out test set

# Frame Extraction Parameters
N_FRAMES_TO_EXTRACT = 16  # Number of frames to extract per video. Adjust as needed.
                            # Too few might miss the sign, too many increases data size.
IMAGE_SAVE_FORMAT = ".jpg" # or ".png"

# Create directories
os.makedirs(BASE_OUTPUT_PATH_FRAMES, exist_ok=True)
os.makedirs(YOLO_FRAME_DATASET_ROOT, exist_ok=True)
os.makedirs(VIDEO_SPLIT_PATH, exist_ok=True)

### 1. Dataset Splitting (Re-using the function from previous script)

In [3]:
def split_video_dataset(
    raw_dataset_path,
    output_split_path,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    random_seed=42,
):
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Ratios must sum to 1"
    random.seed(random_seed)

    if not os.path.exists(raw_dataset_path):
        print(f"Error: Raw dataset path '{raw_dataset_path}' does not exist.")
        return False

    for split in ['train', 'val', 'test']:
        split_folder = os.path.join(output_split_path, split)
        os.makedirs(split_folder, exist_ok=True)

    print(f"Starting dataset split from '{raw_dataset_path}' into '{output_split_path}'...")
    for class_name in os.listdir(raw_dataset_path):
        class_dir = os.path.join(raw_dataset_path, class_name)
        if not os.path.isdir(class_dir):
            continue

        video_files = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f)) and f.lower().endswith(('.mp4', '.avi', '.mov'))]
        if not video_files:
            print(f"Warning: No video files found in class '{class_name}'.")
            continue
        
        random.shuffle(video_files)
        n_total = len(video_files)
        n_train = int(n_total * train_ratio)
        n_val = int(n_total * val_ratio)
        n_test = n_total - n_train - n_val
        if n_test < 0: n_test = 0

        splits_data = {
            'train': video_files[:n_train],
            'val': video_files[n_train : n_train + n_val],
            'test': video_files[n_train + n_val :]
        }
        
        current_assigned_count = len(splits_data['train']) + len(splits_data['val']) + len(splits_data['test'])
        if current_assigned_count < n_total:
            remaining_files = video_files[current_assigned_count:]
            if len(splits_data['test']) < n_test and test_ratio > 0:
                 splits_data['test'].extend(remaining_files)
            elif len(splits_data['val']) < n_val and val_ratio > 0 :
                 splits_data['val'].extend(remaining_files)
            else:
                 splits_data['train'].extend(remaining_files)
        
        for split_name, files_in_split in splits_data.items():
            dest_class_dir = os.path.join(output_split_path, split_name, class_name)
            os.makedirs(dest_class_dir, exist_ok=True)
            for file_name in files_in_split:
                src_file = os.path.join(class_dir, file_name)
                dest_file = os.path.join(dest_class_dir, file_name)
                shutil.copy2(src_file, dest_file) # Always copy for this frame extraction pipeline
        
        print(f"Class '{class_name}' split: Train={len(splits_data['train'])}, Val={len(splits_data['val'])}, Test={len(splits_data['test'])}")
    print("✅ Video dataset splitting complete.")
    return True

### 2. Frame Extraction

In [4]:
def extract_frames_from_video(video_path, num_frames_to_extract):
    """
    Extracts a specified number of evenly spaced frames from a video.
    Returns a list of frames (numpy arrays).
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []

    if total_frames == 0:
        cap.release()
        print(f"Warning: Video {video_path} has 0 frames.")
        return []
        
    if total_frames < num_frames_to_extract:
        # If fewer frames than desired, take all available frames
        # And duplicate the last frame to meet num_frames_to_extract
        print(f"Warning: Video {video_path} has {total_frames} frames, less than desired {num_frames_to_extract}. Taking all and padding.")
        frame_indices = list(range(total_frames))
    else:
        # Select evenly spaced frames
        frame_indices = np.linspace(0, total_frames - 1, num_frames_to_extract, dtype=int)

    collected_frames_count = 0
    last_frame = None
    for i in sorted(list(set(frame_indices))): # Use set to avoid duplicate indices from linspace if num_frames_to_extract > total_frames
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
            last_frame = frame
            collected_frames_count +=1
        else:
            # This might happen if cap.set fails near the end of video
            print(f"Warning: Could not read frame {i} from {video_path}, even though total_frames={total_frames}")
            if last_frame is not None: # Duplicate last good frame if we can't read
                frames.append(last_frame.copy()) 
            else: # If no frames read yet and one fails, this is an issue
                break 


    # Pad with the last frame if we still don't have enough (e.g. if total_frames was very small)
    while len(frames) < num_frames_to_extract and last_frame is not None:
        frames.append(last_frame.copy())
    
    # If no frames were extracted at all and we need some
    if not frames and num_frames_to_extract > 0:
        print(f"Critical Warning: No frames extracted from {video_path}. Returning empty list.")


    cap.release()
    return frames[:num_frames_to_extract] # Ensure exact number


def process_video_splits_for_frames(video_split_dir_root, yolo_output_root, num_frames_per_video):
    """
    Iterates through train/val/test splits of videos, extracts frames,
    and saves them into the YOLO classification dataset structure.
    """
    for split_name in ['train', 'val', 'test']:
        current_video_split_dir = os.path.join(video_split_dir_root, split_name)
        yolo_target_split_dir = os.path.join(yolo_output_root, split_name)
        os.makedirs(yolo_target_split_dir, exist_ok=True)

        if not os.path.exists(current_video_split_dir):
            print(f"Video split directory not found: {current_video_split_dir}. Skipping.")
            continue
        
        print(f"\nProcessing videos in: {current_video_split_dir} for YOLO '{split_name}' set.")
        for class_name in os.listdir(current_video_split_dir):
            video_class_dir = os.path.join(current_video_split_dir, class_name)
            yolo_target_class_dir = os.path.join(yolo_target_split_dir, class_name)
            os.makedirs(yolo_target_class_dir, exist_ok=True)

            if not os.path.isdir(video_class_dir):
                continue

            video_files = [f for f in os.listdir(video_class_dir) if f.lower().endswith(('.mp4', '.avi', '.mov'))]
            for video_file in tqdm(video_files, desc=f"Extracting frames for {class_name} ({split_name})"):
                video_full_path = os.path.join(video_class_dir, video_file)
                video_base_name = os.path.splitext(video_file)[0]

                extracted_frames = extract_frames_from_video(video_full_path, num_frames_per_video)

                for i, frame_img in enumerate(extracted_frames):
                    frame_filename = f"{video_base_name}_frame_{i:03d}{IMAGE_SAVE_FORMAT}"
                    frame_output_path = os.path.join(yolo_target_class_dir, frame_filename)
                    try:
                        cv2.imwrite(frame_output_path, frame_img)
                    except Exception as e:
                        print(f"Error saving frame {frame_output_path}: {e}")
    
    print(f"\n✅ Frame extraction for YOLOv8 classification dataset complete at: {yolo_output_root}")

### Main Execution

In [ ]:
if __name__ == "__main__":
    # Step 1: Split the raw video dataset
    print("--- STEP 1: Splitting Raw Video Dataset ---")
    # Check if video split path already exists and has content (e.g., train folder)
    if not (os.path.exists(os.path.join(VIDEO_SPLIT_PATH, "train")) and \
            os.listdir(os.path.join(VIDEO_SPLIT_PATH, "train"))):
        split_success = split_video_dataset(RAW_VIDEO_DATASET_PATH, VIDEO_SPLIT_PATH, TRAIN_RATIO, VAL_RATIO, TEST_RATIO)
        if not split_success:
            print("Halting due to error in video dataset splitting.")
            exit()
    else:
        print(f"Video splits already found at '{VIDEO_SPLIT_PATH}'. Skipping video splitting.")

    # Step 2: Extract frames from the split videos and save in YOLO format
    print("\n--- STEP 2: Extracting Frames for YOLOv8 Classification ---")
    # Check if YOLO dataset dir exists and has content (e.g., train folder)
    if not (os.path.exists(os.path.join(YOLO_FRAME_DATASET_ROOT, "train")) and \
            os.listdir(os.path.join(YOLO_FRAME_DATASET_ROOT, "train"))):
        process_video_splits_for_frames(VIDEO_SPLIT_PATH, YOLO_FRAME_DATASET_ROOT, N_FRAMES_TO_EXTRACT)
    else:
        print(f"YOLO frame dataset already found at '{YOLO_FRAME_DATASET_ROOT}'. Skipping frame extraction.")
    
    print("\n--- Preprocessing for YOLOv8-cls Complete ---")

    ## Training YOLOv8-cls Model

In [ ]:
%pip install ultralytics

In [7]:
from ultralytics import YOLO
import os

In [8]:
YOLO_DATASET_ROOT = os.path.join("SignLanguage_Processed_Frames", "YOLO_Frames_Dataset")
MODEL_TYPE = 'yolov8n-cls.pt'  # Start with nano, can try 'yolov8s-cls.pt', etc.
EPOCHS = 100  # Number of training epochs
IMG_SIZE = 224 # Standard image size for classification models, YOLO will resize
BATCH_SIZE = 32 # Adjust based on your GPU memory
PROJECT_NAME = 'runs/slr_yolo_frame_cls_train'
EXPERIMENT_NAME = 'exp_frames'


In [ ]:
def main():
    if not os.path.exists(YOLO_DATASET_ROOT):
        print(f"ERROR: Dataset directory not found: {YOLO_DATASET_ROOT}")
        print("Please run the 'preprocess_frames_for_yolov8_cls.py' script first.")
        return

    # Load a pretrained YOLOv8-cls model
    model = YOLO(MODEL_TYPE)

    # Train the model
    print(f"\n--- Starting YOLOv8-cls Training ---")
    print(f"Dataset: {YOLO_DATASET_ROOT}")
    print(f"Model: {MODEL_TYPE}")
    print(f"Epochs: {EPOCHS}")
    print(f"Image Size: {IMG_SIZE}")
    print(f"Batch Size: {BATCH_SIZE}")

    results = model.train(
        data=YOLO_DATASET_ROOT,
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        patience=10,  # Early stopping patience
        workers=4,    # Number of dataloader workers (adjust based on your CPU)
        project=PROJECT_NAME,
        name=EXPERIMENT_NAME,
        # device='0' # Specify GPU device if you have multiple, otherwise auto-detect
    )

    print(f"\n--- Training Complete ---")
    print(f"Results saved in: {os.path.join(PROJECT_NAME, EXPERIMENT_NAME)}")

    # Optional: Evaluate on the validation set explicitly
    print("\n--- Evaluating on Validation Set ---")
    val_results = model.val()
    print("Validation Metrics:", val_results.results_dict) # Or explore other attributes of val_results

if __name__ == '__main__':
    main()